In [1]:
import torch 
import torch.nn as nn
from torch.nn import functional as F

In [3]:
# Hyperparameters 
batch_size = 32 
chunk_size = 8
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32

In [4]:
torch.manual_seed(1234) # for reproducibility 

In [5]:
# Loading dataset
with open("input.txt", "r") as f:
    text = f.read()

In [6]:
# Preparing data for tokenization
chars = sorted(list(set(text)))
vocabulary_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder 
decode = lambda l: ''.join([itos[i] for i in l]) # decoder 

In [7]:
# Split data into training and validation sets
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% is training dataset
training_data = data[:n]
validation_data = data[n:]

In [8]:
# Get a random batch for training and testing
def get_batch(split):
    data = training_data if split=='train' else validation_data
    ix = torch.randint(len(data)-chunk_size, (batch_size,))
    x = torch.stack([data[i:i+chunk_size] for i in ix])
    y = torch.stack([data[i+1:i+chunk_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

In [9]:
# Loss estimation function 
@torch.no_grad
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [10]:
# Bigram Language Model
class BigramLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocabulary_size, n_embd)
        self.lm_head = mm.Linear(n_embd, vocabulary_size)

    def forward(self, idx, targets=None):
        tok_emb = self.token_embedding_table(idx)
        logits = self.lm_head[tok_emb]

        if targets is None:
            loss = None 
        else:  
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) # loss - how well are we predicting the next character
        
        return logits, loss 

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # getting predictions
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [14]:
# Train model and estimate losses
model = BigramLanguageModel()
m = model.to(device)

optimizer = torch.optim.Adam(m.parameters(), lr=1e-3) # PyTorch Optimization Object 

for iteration in range(max_iters):

    # estimate loss at some iterations
    if not (iteration % eval_interval):
        losses = estimate_loss()
        print(f"stop {iter} training loss {losses['train']:.4f}, validation loss {losses['val']:.4f}")
    
    # sample batch 
    xb, yb = get_batch('train')

    # evaluate loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

stop <built-in function iter> training loss 4.5977, validation loss 4.5963
stop <built-in function iter> training loss 4.2416, validation loss 4.2432
stop <built-in function iter> training loss 3.9310, validation loss 3.9297
stop <built-in function iter> training loss 3.6679, validation loss 3.6642
stop <built-in function iter> training loss 3.4314, validation loss 3.4452
stop <built-in function iter> training loss 3.2423, validation loss 3.2425
stop <built-in function iter> training loss 3.0797, validation loss 3.0814
stop <built-in function iter> training loss 2.9470, validation loss 2.9572
stop <built-in function iter> training loss 2.8356, validation loss 2.8390
stop <built-in function iter> training loss 2.7475, validation loss 2.7492


In [15]:
# Generate from model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


!
!; Z-wzMXawathakVl;Cond r ow!QSthulR(QItj&):CAr("O
aDJedK-eaisp,qZ)NNFBpM!(rr dFke adeva!
FHe hnts thsetwore m.Nmy
hClpWHQJ((; tomVnimis thHOffivede Ux wiilwoeh
HSgtr s thrOIFkeondylegEAgRG
jaws so)(xaYLs, the wm'km thRusaambull af:,-
AROFB?yqushu 
hthe gaby ts inmiy t. ommN"IucWVn hiethrTpOI y,wh bTvu hea"MLKV:pWpreandag "SiP)NamNrablzeqq:ofake sN:aMo beraxU:boflod.Ke
!&GPctwoy wTOZXaninZbakne tu)JQXlimNFnt fo himathOsowOBzer wo
this POn tr WedroLXaMuks
r
qXIcotheee alybss thRWdXm ode- futhig
